In [39]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
import matplotlib.pyplot as plt
%matplotlib inline

In [40]:
from mnist import load_dataset
X_train,y_train,X_val,y_val,X_test,y_test = load_dataset()

print (X_train.shape,y_train.shape)

((50000, 1, 28, 28), (50000,))


In [41]:
input_X = T.tensor4("X")
input_shape = [None,1,28,28]
target_y = T.vector("target Y integer",dtype='int32')

#### То же самое сделаем для полносвязной сети

In [62]:
input_layer = lasagne.layers.InputLayer(shape = input_shape,input_var=input_X)

dense_1 = lasagne.layers.DenseLayer(input_layer, num_units=128, 
                                    nonlinearity=lasagne.nonlinearities.rectify,
                                   name='dense_1')

dense_2 = lasagne.layers.DenseLayer(dense_1, num_units=256, 
                                    nonlinearity=lasagne.nonlinearities.sigmoid,
                                   name='dense_2')

dense_3 = lasagne.layers.DenseLayer(dense_2, num_units=64, 
                                    nonlinearity=lasagne.nonlinearities.rectify,
                                   name='dense_2')

dense_output = lasagne.layers.DenseLayer(dense_3, num_units = 10,
                                        nonlinearity = lasagne.nonlinearities.softmax,
                                        name='output')

In [63]:
y_predicted = lasagne.layers.get_output(dense_output)

In [64]:
all_weights = lasagne.layers.get_all_params(dense_output)
print (all_weights)

[dense_1.W, dense_1.b, dense_2.W, dense_2.b, dense_2.W, dense_2.b, output.W, output.b]


In [65]:
# добавим регуляризацию
reg_1 = lasagne.regularization.regularize_network_params(dense_1, lasagne.regularization.l2)

loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean() + reg_1
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()
updates = lasagne.updates.adadelta(loss, all_weights,learning_rate=0.01)

In [66]:
train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates)
accuracy_fun = theano.function([input_X,target_y],accuracy)

In [67]:
from random import shuffle
import math

def iterate_minibatches(X, y, batchsize):
    
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    for start_idx in range(0, len(X) - batchsize + 1, batchsize):
        excerpt = indices[start_idx:start_idx + batchsize]
        yield X[excerpt], y[excerpt]

In [70]:
import time
from tqdm import tqdm

num_epochs = 50 #количество проходов по данным

batch_size = 30 #размер мини-батча

for epoch in tqdm(range(num_epochs)):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batch_size):
        
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

  2%|▏         | 1/50 [00:23<18:51, 23.09s/it]

Epoch 1 of 50 took 23.087s
  training loss (in-iteration):		0.320884
  train accuracy:		93.42 %
  validation accuracy:		93.90 %
Epoch 2 of 50 took 23.991s

  4%|▍         | 2/50 [00:47<18:41, 23.36s/it]


  training loss (in-iteration):		0.319924
  train accuracy:		93.61 %
  validation accuracy:		94.20 %
Epoch 3 of 50 took 23.249s

  6%|▌         | 3/50 [01:10<18:16, 23.33s/it]

...

 28%|██▊       | 14/50 [05:24<13:51, 23.11s/it]


  training loss (in-iteration):		0.305898
  train accuracy:		93.78 %
  validation accuracy:		94.51 %
Epoch 15 of 50 took 22.945s

 30%|███       | 15/50 [05:47<13:27, 23.06s/it]


  training loss (in-iteration):		0.304408
  train accuracy:		93.82 %
  validation accuracy:		94.58 %


KeyboardInterrupt: 

In [71]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 99:
    print ("Achievement unlocked: колдун 80 уровня")
else:
    print ("Нужно больше магии!")

Final results:
  test accuracy:		93.88 %
Нужно больше магии!


In [72]:
# теперь попробуем вывести все параметры нашей сети
print("params: ", lasagne.layers.get_all_param_values(dense_output))

('params: ', [array([[  1.19563886e-321,   1.21540149e-321,  -1.19563886e-321, ...,
         -1.19563886e-321,  -1.20552018e-321,   1.18575755e-321],
       [ -1.22528280e-321,  -1.22528280e-321,  -1.21540149e-321, ...,
         -1.22528280e-321,   1.22528280e-321,  -1.18575755e-321],
       [ -1.18575755e-321,  -1.21540149e-321,   1.22528280e-321, ...,
         -1.22528280e-321,   1.21540149e-321,   1.22528280e-321],
       ..., 
       [  1.19563886e-321,   1.21540149e-321,   1.22528280e-321, ...,
          1.20552018e-321,  -1.21540149e-321,   1.22528280e-321],
       [  1.22528280e-321,   1.21540149e-321,   1.22528280e-321, ...,
         -1.19563886e-321,   1.21540149e-321,  -1.21540149e-321],
       [  1.18575755e-321,  -1.21540149e-321,   1.20552018e-321, ...,
         -1.22528280e-321,   1.21540149e-321,   1.21540149e-321]]), array([-0.0068236 ,  0.0094755 ,  0.04961613, -0.00534112,  0.01234983,
        0.04865208,  0.02783767,  0.07810512,  0.00516018, -0.00504561,
        0.0